In [1]:
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import datetime
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from scipy.stats import qmc

import bo_methods_lib
from bo_methods_lib.bo_functions_generic import eval_GP_sparse_grid, LHS_Design, normalize_bounds, gen_theta_set, find_train_doc_path, test_train_split, clean_1D_arrays, norm_unnorm, calc_ei_basic, calc_ei_emulator
from bo_methods_lib.normalize import normalize_p_data, normalize_x, normalize_p_true, normalize_p_bounds
from bo_methods_lib.CS2_create_data import *
from bo_methods_lib.CS2_bo_functions_multi_dim import argmax_multiple
# from bo_methods_lib.CS1_create_data import *
# from bo_methods_lib.test_CS1_create_data import *
import itertools
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations
from sklearn.preprocessing import MinMaxScaler
import random

# import matplotlib as mpl
# mpl.rcParams['figure.dpi'] = 300

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Meshgrid to combinations
p=3
x_space = gen_theta_set(LHS = False, n_points = p, dimensions = 2, bounds = bounds_x)
print(x_space.reshape(p,p,-1).T)

X1 =  np.linspace(-1.5,1,p) #1x10
X2 =  np.linspace(-0.5,2,p) #1x10
X_mesh = np.array(np.meshgrid(X1, X2)) #2 Uniform 5x5 arrays
# print(X_mesh)

Test1 = np.zeros((X_mesh.shape[1:3]))
for i in range(X_mesh.shape[1]):
    #Loop over X2
    for j in range(X_mesh.shape[2]):
        Test1[i,j] = X_mesh[0,i,j] + X_mesh[1,i,j]

print(Test1)

Test2 = []
for k in range(len(x_space)):
    Test2.append(x_space[k,0] + x_space[k,1])
Test2 = np.array(Test2).reshape((p, p))

print(Test2)

In [8]:
EI_max = np.load("2022/10/21/13-18/CSV_Data/Convergence_Figs/GP_Emulator/Approx/TP_100/obj/ep_0.8/len_scl_varies/Sep_Fact_1.0/Total_Runs_15/Max_EI_Conv.npy")
EI_max = np.load("2022/10/21/13-18/CSV_Data/Convergence_Figs/GP_Error_Emulator/TP_20/obj/ep_0.3/len_scl_varies/Sep_Fact_1.0/Total_Runs_15/Max_EI_Conv.npy")
print(np.max(EI_max[:,99]))

5.077154966394979e-05


In [30]:
CS = 1

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])
if CS == 2.2:
    skip_param_types = 1 #This is what changes for subpoint
    Theta_True = Constants[skip_param_types:skip_param_types+2].flatten()
    param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
                  4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}
    exp_d = 2
    n = 15 #Number of experimental data points to use
    bounds_p = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                       [ 2,  2,   0,  2,  2,  2, 15,  2]])
    bounds_x = np.array([[-1.5,-0.5],[1,2]])
    LHS = True

else:
    Constants = Theta_True = np.array([1,-1])
    skip_param_types = 0
    param_dict = {0 : '\\theta_1', 1 : '\\theta_2'}
    exp_d = 1
    bounds_x = np.array([[-2],[2]])
    bounds_p = np.array([[-2,-2],
                     [2 , 2]])
    LHS = False
    n = 5

print(Constants.shape)
print(Theta_True.shape)

p = 20
d = len(Theta_True)

obj = np.array(["obj, LN_obj"])
emulator = np.array([True, False])

# if emulator == True:
#     t = 20*n
# else:
#     t = 20

# #Pull Experimental data from CSV
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]
Xexp = clean_1D_arrays(Xexp)

m = Xexp.shape[1]
print(Xexp.shape)

theta_mesh = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds_p)
print(theta_mesh.shape)
theta_set = theta_mesh

# all_data_doc = find_train_doc_path(emulator, obj, d, t)
# all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=",")) 
# print(all_data.shape)

# train_data, test_data = test_train_split(all_data, sep_fact = 1.0, shuffle_seed=9)

# if CS == 1 and emulator == False:
#     train_p, test_p = train_data[:,1:-2], test_data[:,1:-2]
# else:
#     train_p, test_p = train_data[:,1:-1], test_data[:,1:-1]
# print(train_p)
print(train_p.shape)
print(test_p.shape)

(2,)
(2,)
(5, 1)
(400, 2)
(100, 3)
(0, 3)


In [48]:
DataFrame.to_csv(

          Theta 1     Theta 2        Xexp   Obj Value
count  100.000000  100.000000  100.000000  100.000000
mean    -0.002491   -0.024375    0.000000   -0.048750
std      1.162189    1.156825    1.421338    6.166969
min     -1.976573   -1.955507   -2.000000  -19.180768
25%     -1.005618   -0.994930   -1.000000   -2.116313
50%      0.055788   -0.057137    0.000000    0.000000
75%      0.987064    0.944770    1.000000    1.583727
max      1.815352    1.987549    2.000000   18.672163
         Theta 1    Theta 2   Obj Value
count  20.000000  20.000000   20.000000
mean   -0.002491  -0.024375  100.932282
std     1.186404   1.180928   91.766412
min    -1.976573  -1.955507    2.822507
25%    -1.005618  -0.994930   19.294443
50%     0.055788  -0.057137   84.574353
75%     0.987064   0.944770  151.704033
max     1.815352   1.987549  305.491371
         Theta 1    Theta 2  Obj Value
count  20.000000  20.000000  20.000000
mean   -0.002491  -0.024375   3.962640
std     1.186404   1.180928   1.40200

In [2]:
argmax= np.array([0,1,2,3])
train_p = np.array([[-1.5, -1.5], [-1.6, -1.6], [-1.7, -1.7], [-1.8, -1.8]])
theta_set = np.array([[1.5, 1.5], [1.6, 1.6], [1.7, 1.7], [1.8, 1.8]])
print(argmax_multiple(argmax, train_p, theta_set))

[3]


In [ ]:
#Calculating actual SSE Values
norm = True
bounds_x_scl, scaler_x = normalize_x(bounds_x, None, norm, scaler = None)
bounds_p_scl, scaler_theta = normalize_p_bounds(bounds_p, norm)
Xexp_scl = normalize_x(Xexp, None, norm, scaler = scaler_x)[0]
print(Xexp_scl)
Xexxp = Xexp_scl[0]
print(Xexp[0])
print(normalize_x(clean_1D_arrays(Xexxp, param_clean = True), None, False, scaler_x))

Guess_0 = np.array([0.61032438,0.43428296,0.72599213,0.66055616,0.52820981,0.95271155,0.94534902,0.87020441])
Guess_0 = normalize_p_true(Guess_0, scaler_theta, norm = False)
Guess_1 = np.array([0,1,0.370735521, 9.23236422E-04,0.166088842, 0.169900862,1,0.024381367])
Guess_1 = normalize_p_true(Guess_1, scaler_theta, norm = False)
Guess_2 = Theta_True



SSE_guess_0 = create_sse_data(Guess_0, Xexp, Yexp, Constants, obj = "obj", skip_param_types = 1)
# GP_guess_0 = 1236476096410.422
GP_guess_0 = 74323.09891172957
SSE_guess_1 = create_sse_data(Guess_1, Xexp, Yexp, Constants, obj = "obj", skip_param_types = 1)
GP_guess_1 =-945327999181.6357
SSE_guess_2 = create_sse_data(Guess_2, Xexp, Yexp, Constants, obj = "obj", skip_param_types = 1)
print(Guess_0, Guess_1)
print("True Params \n", Theta_True)
print("True SSE \n", float(SSE_guess_2))
print("Guess 1 \n", np.round(Guess_0,3))
print("True SSE Guess 1 \n", float(SSE_guess_0), "\n GP Guess 1 \n", GP_guess_0, "\n difference %", abs((GP_guess_0 - float(SSE_guess_0))/float(SSE_guess_0)))
# print("Guess 2 \n", np.round(Guess_1,3))
# print("True SSE Guess 2 \n", float(SSE_guess_1), "\n GP Guess 2 \n", GP_guess_1, "\n difference %", abs((GP_guess_1 - float(SSE_guess_1))/float(SSE_guess_1)))
# print(SSE_guess_2, GP_guess_2)

In [ ]:
# https://realpython.com/python-timer/
import time
start = 3678393
res = datetime.timedelta(seconds =start)
print(res)

time_tot = 0
x = 0
for i in range(89900):
    start2 = time.time()
    x += 1+2
    end = time.time()

    time_tot+= (end - start2)


print(time_tot)